<a href="https://colab.research.google.com/github/eshika289/cs301_project/blob/main/CS_301_NLP_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Statements

In [91]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import jax

https://jax.readthedocs.io/en/latest/notebooks/quickstart.html

In [256]:
import os, shutil
import csv
import re
import math
import random
import numpy as np
from collections import Counter
from collections import defaultdict

In [79]:
from typing import List, Union
from typing import Any
from typing import Dict
from typing import TypeVar
from typing import NamedTuple, Optional

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Get Data From Kaggle

Get kaggle.json file from kaggle and upload it to the content folder

In [92]:
!pwd # /content

/content


In [93]:
!mkdir /root/.kaggle
!mkdir kaggle

In [ ]:
!mv ./kaggle.json /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json

In [96]:
#!pip install kaggle
!kaggle -v

Kaggle API 1.5.12


In [97]:
!chmod 600 /root/.kaggle/kaggle.json

In [98]:
!kaggle competitions files -c nlp-getting-started

name                    size  creationDate         
---------------------  -----  -------------------  
train.csv              965KB  2019-12-16 20:36:20  
test.csv               411KB  2019-12-16 20:36:20  
sample_submission.csv   22KB  2019-12-16 20:36:20  


In [99]:
!kaggle competitions download -c nlp-getting-started -p /content/kaggle/

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 104MB/s]


In [360]:
!unzip -q /content/kaggle/nlp-getting-started.zip -d /content/kaggle/data

replace /content/kaggle/data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace /content/kaggle/data/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace /content/kaggle/data/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [394]:
data_path = "kaggle/data/train.csv"
data_file = open(data_path)
read_file = csv.reader(data_file)
headers = []
headers = next(read_file)
data = []
for row in read_file:
  data.append(row)

In [395]:
print(headers) # ['id', 'keyword', 'location', 'text', 'target']
print(data[0])
print(len(data))

['id', 'keyword', 'location', 'text', 'target']
['1', '', '', 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', '1']
7613


# Features

## Most Common Words as Features

In [ ]:
def most_common_words(subset):
  all_text = ""
  for i in range(len(subset)):
    all_text+=subset[i][3]+" "

  stop_words = stopwords.words('english')
  all_text.lower()
  words_list = all_text.split()
  word_list = []
  part_of_speech = {}

  for word in words_list:
    if word not in stop_words and re.search('[a-z]', word) is not None and word!="&amp;" and word[:4]!="http":
      #tmp = wn.synsets(word)[0].pos()
      word_list.append(word.lower())
      #part_of_speech[word.lower]= tmp

  #words_dict = dict(sorted(words_dict.items(), key=lambda item: -item[1]))
  words_dict = Counter(word_list)
  for item in words_dict.most_common(50):
    print(item)
    #print(part_of_speech[item])

most_common_words(data)

## Bigrams / Trigrams As Features

## Grammars (nouns,verbs) As Features

# Decision Tree

## Entropy

In [343]:
def entropy(class_probabilities: List[float]) -> float:
  '''Given a list of class probabilities, compute the entropy'''
  return jnp.sum(jnp.asarray([-p*jnp.log2(p) for p in class_probabilities if p > 0]))

def class_probabilities(labels: List[Any]) -> float:
  total_count = jnp.size(labels)
  return [count/total_count for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
  return entropy(class_probabilities(labels))

def partition_entropy(subsets: List[List[Any]]) -> float:
  '''Returns the entropy from this partition of data into subsets'''
  total_count = jnp.sum(jnp.asarray([jnp.size(subset) for subset in subsets]))
  return jnp.sum(jnp.asarray([data_entropy(subset)*jnp.size(subset)/total_count for subset in subsets]))

## Parition

In [304]:
T = TypeVar('T')

def partition_by(inputs: List[T], attribute:str) -> Dict[Any, List[T]]:
  '''Partitions the inputs into lists based on the specified attribute'''
  partitions: Dict[Any,List[T]] = defaultdict(list)
  for input in inputs:
    key = getattr(input, attribute)
    partitions[key].append(input)
  return partitions

def partition_entropy_by(inputs: List[Any], attribute:str,label_attribute:str) ->float:
  '''Computes the entropy corresponding to the given partition'''
  partitions = partition_by(inputs,attribute)
  labels = [[getattr(input, label_attribute) for input in partition] for partition in partitions.values()]
  return partition_entropy(labels)

## Test Entropy and Partition

In [103]:
print(headers)

['id', 'keyword', 'location', 'text', 'target']


In [389]:
tweets = []

class Tweet(NamedTuple):
  keyword: str
  location: str
  target: int

for tweet in train_data:
  tweets.append(Tweet(tweet[1],tweet[2],tweet[4]))

In [390]:
print(len(tweets))
print(len(train_data))

5000
5000


In [375]:
for key in ['keyword','location']:
  print(key,partition_entropy_by(tweets,key,'target'))

keyword 0.7342361
location 0.5156009


## Build Tree

In [80]:
class Leaf(NamedTuple):
  value: Any

class Split(NamedTuple):
  attribute: str
  subtrees: dict
  default_value: Any = None

DecisionTree = Union[Leaf,Split]

In [391]:
class Tweet(NamedTuple):
  keyword: str
  location: str
  target: int

In [81]:
def classify(tree:DecisionTree,input:Any) -> Any:
  '''Classify the input using the given decision tree'''
  
  # If this is a leaf node, return its value
  if isinstance(tree,Leaf):
    return tree.value

  subtree_key = getattr(input, tree.attribute)

  if subtree_key not in tree.subtrees:
    return tree.default_value
  
  subtree = tree.subtrees[subtree_key]
  return classify(subtree,input)

In [84]:
def build_tree_id3(inputs: List[Any],split_attributes:List[str],target_attribute:str)->DecisionTree:
  # Count target labels
  label_counts = Counter(getattr(input,target_attribute) for input in inputs)
  most_common_label = label_counts.most_common(1)[0][0]

  # If there's a unique label, predict it
  if len(label_counts) == 1:
    return Leaf(most_common_label)
  
  # If no split attributes left, return the majority label
  if not split_attributes:
    return Leaf(most_common_label)

  # Otherwise split by the best attribute
  def split_entropy(attribute:str) -> float:
    return partition_entropy_by(inputs,attribute,target_attribute)
  
  best_attribute = min(split_attributes, key=split_entropy)

  partitions = partition_by(inputs, best_attribute)
  new_attributes = [a for a in split_attributes if a !=best_attribute]

  # Recursively build subtrees
  subtrees = {attribute_value: build_tree_id3(subset,new_attributes, target_attribute) for attribute_value,subset in partitions.items()}

  return Split(best_attribute, subtrees, default_value=most_common_label)

## Test Build Tree

In [381]:
decision_tree = build_tree_id3(tweets,['keyword','location'],'target')
print(decision_tree)

Split(attribute='location', subtrees={'': Split(attribute='keyword', subtrees={'razed': Leaf(value='1'), 'blaze': Leaf(value='0'), 'drought': Leaf(value='1'), 'natural%20disaster': Leaf(value='1'), 'refugees': Leaf(value='1'), 'bleeding': Leaf(value='0'), 'devastation': Leaf(value='1'), 'quarantine': Leaf(value='0'), 'burned': Leaf(value='0'), 'harm': Leaf(value='0'), 'bridge%20collapse': Leaf(value='1'), 'hurricane': Leaf(value='1'), 'suicide%20bomber': Leaf(value='1'), 'accident': Leaf(value='0'), 'dead': Leaf(value='0'), 'demolition': Leaf(value='0'), 'bioterror': Leaf(value='1'), 'typhoon': Leaf(value='1'), 'riot': Leaf(value='0'), 'bombed': Leaf(value='0'), 'panic': Leaf(value='0'), 'body%20bag': Leaf(value='0'), 'heat%20wave': Leaf(value='0'), 'fatal': Leaf(value='1'), 'police': Leaf(value='0'), 'ruin': Leaf(value='0'), 'trapped': Leaf(value='0'), 'burning': Leaf(value='0'), '': Leaf(value='1'), 'siren': Leaf(value='0'), 'hijacking': Leaf(value='1'), 'fear': Leaf(value='0'), 'exp

## Accuracy of Tree

In [482]:
def accuracy(decision_tree,tweets):
  correct = 0
  incorrect = 0
  for tweet in tweets:
    predictedOutput(decision_tree,tweet)
    if value== tweet.target:
      correct+=1
    else:
      incorrect+=1
  #print("Correct: ", correct)
  #print("Incorrect: ", incorrect)

  accuracy = correct/(correct+incorrect)
  #print("Accuracy: ", accuracy)
  return accuracy


def predictedOutput(decision_tree,tweet):
  ''' will use the decision_tree to predict if tweet is valid or not
      changes the global variable value to 1 for valid and 0 for invalid
  '''
  global value
  tree= decision_tree
  attribute = tree[0]
  tweet_val = None
  if attribute == 'location':
    tweet_val = tweet.location
  if attribute == 'keyword':
    tweet_val = tweet.keyword
  try:
    tree = tree[1][tweet_val]
    if isinstance(tree,Leaf):
        value = tree.value
    else:
      predictedOutput(tree,tweet)
  except:
    value = tree[2]

In [483]:
accuracy(decision_tree,tweets)
#predictedOutput(decision_tree,tweets[0])

0.9218

## Test Dataset Accuracy

In [463]:
test_tweets = []
for tweet in test_data:
  test_tweets.append(Tweet(tweet[1],tweet[2],tweet[4]))

print(len(test_data))
print(len(test_tweets))

2613
2613


In [484]:
accuracy(decision_tree,test_tweets)

0.8143895905089935

## Functions to Use to Build Trees 



*   tree(train_data,test_data) -> returns a decision tree
*   predictedOutput(tree,tweet) -> changes the global variable value to 1 if valid and 0 if not valid
*   decision_tree_accuracies(tree,train_tweets,test_tweets) -> returns the train and test accuracies as a tuple (train_accuracy,test_accuracy)


In [496]:
def tree(train_data,test_data):
  global train_tweets
  global test_tweets
  train_tweets = []
  test_tweets = []

  for tweet in train_data:
    train_tweets.append(Tweet(tweet[1],tweet[2],tweet[4]))

  for tweet in test_data:
    test_tweets.append(Tweet(tweet[1],tweet[2],tweet[4]))

  return build_tree_id3(train_tweets,['keyword','location'],'target')

In [497]:
tree = tree(train_data,test_data)

predictedOutput(tree,tweets[100])
print("Predicted Value: ",value)
print("Actual Value: ", tweets[40].target)

predictedOutput(tree,tweets[40])
print("Predicted Value: ",value)
print("Actual Value: ", tweets[40].target)

Predicted Value:  0
Actual Value:  1
Predicted Value:  1
Actual Value:  1


In [498]:
def decision_tree_accuracies(tree,train_tweets,test_tweets):


  one = accuracy(tree,train_tweets)
  two = accuracy(decision_tree,test_tweets)

  return(one,two)


In [499]:
train_acc,test_acc = (decision_tree_accuracies(tree,train_tweets,test_tweets))
print("train acc:",train_acc)
print("test acc:",test_acc)

train acc: 0.918
test acc: 0.8396479142747799


# Random Forests

## Shuffle and Split Data

https://jax.readthedocs.io/en/latest/_autosummary/jax.random.shuffle.html

In [494]:
def shuffle(array):
  array = np.random.permutation(array)
  return array

def split_data(data):
  global test_data, train_data
  test_data = data[5000:]
  train_data = data[:5000]

def shuffle_and_split(data):
  data = shuffle(data)
  split_data(data)
  
shuffle_and_split(data)